# Quick Demos of Statistics and Machine Learning in Python

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_rows = 10
%matplotlib inline

## Dealing with categorical data

In [ ]:
np.random.seed(27)
s = pd.Series(np.random.choice(['bad', 'neutral', 'good'], size=40))
s

In [ ]:
np.random.seed(27)
s = pd.Series(pd.Categorical(np.random.choice(['bad', 'neutral', 'good'], size=40),
                             categories=['bad', 'neutral', 'good'], ordered=True))
s

Categoricals can go inside of Series or DataFrames just like any other column. The have a
a special `.cat` namespace.

In [ ]:
s.cat.categories

In [ ]:
s.cat.codes

In [ ]:
df = pd.DataFrame({'a': np.arange(40), 'b': s})
df

### Dummy variables

In [ ]:
pd.get_dummies(s)

In [ ]:
kinds = [
    'A|B',
    'A|B|C',
    'C',
    'B|A',
    'A|B'
]
s = pd.Series(kinds)
s

In [ ]:
s.str.get_dummies(sep='|')

# Statsmodels

Brief primer: you give a **estimator** (OLS, WLS, GLM) a **formula** and **dataset**. You then fit that model.

In [ ]:
import statsmodels.api as sm

We've got some longintudinal data (repeated measures of the same individual on children with HIV). 

There are two treatment arms (ARV vs control) represented by `treatment`, and administration of the treatment is indicated by `arv`.

In [ ]:
df = pd.read_csv('../data/random/cd4.csv', parse_dates=['VDATE'], index_col=['newpid', 'VISIT'])
df = df.dropna()  # estimator can't handle NaNs so ignore for now.
df.head()

Some basic statistics:

In [ ]:
df.groupby(['arv', 'treatmnt'])['CD4PCT'].count().unstack()

In [ ]:
df.groupby('arv')['CD4CNT'].mean()

In [ ]:
sns.factorplot(x='arv', y='CD4CNT', col='treatmnt', data=df)

In [ ]:
df.head()

In [ ]:
years_since = df.groupby(level='newpid')['VDATE']\
    .apply(lambda x: (x - x.min()).dt.days / 365)
df['age'] = df['baseage'] + years_since

In [ ]:
sns.lmplot('age', 'CD4CNT', data=df, hue="arv")

### Linear regression

In [ ]:
mod = sm.GLM.from_formula('CD4PCT ~ age + arv + treatmnt', df.reset_index())
result = mod.fit()
result.summary()

### Mixed effects models

In [ ]:
mod2 = sm.MixedLM.from_formula('CD4PCT ~ age + arv*treatmnt', df.reset_index(), groups='newpid')
res2 = mod2.fit()
res2.summary()

# Exercise
1. Use a linear regression to find the relationship between SDI and the death rate in 1 to 4 year olds
2. What happens if you use a random effect on `location_id` to adjust for differences between countries?
3. Are we just picking up a time effect? What happens if you add year to the model?

In [ ]:
data = #

In [ ]:
# Part 1
lm = #

In [ ]:
# Part 2
mixed = #

In [ ]:
# Part 3
mixed_with_year = #

# Scikit-Learn

### Gigantic library of many different kinds of ML and Statistical models

#### [Geospatial SVM example](http://scikit-learn.org/stable/auto_examples/applications/plot_species_distribution_modeling.html)

In [ ]:
from time import time

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets.base import Bunch
from sklearn.datasets import fetch_species_distributions
from sklearn.datasets.species_distributions import construct_grids
from sklearn import svm, metrics

# if basemap is available, we'll use it.
# otherwise, we'll improvise later...
try:
    from mpl_toolkits.basemap import Basemap
    basemap = True
except ImportError:
    basemap = False

print(__doc__)


def create_species_bunch(species_name, train, test, coverages, xgrid, ygrid):
    """Create a bunch with information about a particular organism

    This will use the test/train record arrays to extract the
    data specific to the given species name.
    """
    bunch = Bunch(name=' '.join(species_name.split("_")[:2]))
    species_name = species_name.encode('ascii')
    points = dict(test=test, train=train)

    for label, pts in points.items():
        # choose points associated with the desired species
        pts = pts[pts['species'] == species_name]
        bunch['pts_%s' % label] = pts

        # determine coverage values for each of the training & testing points
        ix = np.searchsorted(xgrid, pts['dd long'])
        iy = np.searchsorted(ygrid, pts['dd lat'])
        bunch['cov_%s' % label] = coverages[:, -iy, ix].T

    return bunch


def plot_species_distribution(species=("bradypus_variegatus_0",
                                       "microryzomys_minutus_0")):
    """
    Plot the species distribution.
    """
    if len(species) > 2:
        print("Note: when more than two species are provided,"
              " only the first two will be used")

    t0 = time()

    # Load the compressed data
    data = fetch_species_distributions()

    # Set up the data grid
    xgrid, ygrid = construct_grids(data)

    # The grid in x,y coordinates
    X, Y = np.meshgrid(xgrid, ygrid[::-1])

    # create a bunch for each species
    BV_bunch = create_species_bunch(species[0],
                                    data.train, data.test,
                                    data.coverages, xgrid, ygrid)
    MM_bunch = create_species_bunch(species[1],
                                    data.train, data.test,
                                    data.coverages, xgrid, ygrid)

    # background points (grid coordinates) for evaluation
    np.random.seed(13)
    background_points = np.c_[np.random.randint(low=0, high=data.Ny,
                                                size=10000),
                              np.random.randint(low=0, high=data.Nx,
                                                size=10000)].T

    # We'll make use of the fact that coverages[6] has measurements at all
    # land points.  This will help us decide between land and water.
    land_reference = data.coverages[6]

    # Fit, predict, and plot for each species.
    for i, species in enumerate([BV_bunch, MM_bunch]):
        print("_" * 80)
        print("Modeling distribution of species '%s'" % species.name)

        # Standardize features
        mean = species.cov_train.mean(axis=0)
        std = species.cov_train.std(axis=0)
        train_cover_std = (species.cov_train - mean) / std

        # Fit OneClassSVM
        print(" - fit OneClassSVM ... ")
        clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.5)
        clf.fit(train_cover_std)
        print("done.")

        # Plot map of South America
        plt.subplot(1, 2, i + 1)
        if basemap:
            print(" - plot coastlines using basemap")
            m = Basemap(projection='cyl', llcrnrlat=Y.min(),
                        urcrnrlat=Y.max(), llcrnrlon=X.min(),
                        urcrnrlon=X.max(), resolution='c')
            m.drawcoastlines()
            m.drawcountries()
        else:
            print(" - plot coastlines from coverage")
            plt.contour(X, Y, land_reference,
                        levels=[-9999], colors="k",
                        linestyles="solid")
            plt.xticks([])
            plt.yticks([])

        print(" - predict species distribution")

        # Predict species distribution using the training data
        Z = np.ones((data.Ny, data.Nx), dtype=np.float64)

        # We'll predict only for the land points.
        idx = np.where(land_reference > -9999)
        coverages_land = data.coverages[:, idx[0], idx[1]].T

        pred = clf.decision_function((coverages_land - mean) / std)[:, 0]
        Z *= pred.min()
        Z[idx[0], idx[1]] = pred

        levels = np.linspace(Z.min(), Z.max(), 25)
        Z[land_reference == -9999] = -9999

        # plot contours of the prediction
        plt.contourf(X, Y, Z, levels=levels, cmap=plt.cm.Reds)
        plt.colorbar(format='%.2f')

        # scatter training/testing points
        plt.scatter(species.pts_train['dd long'], species.pts_train['dd lat'],
                    s=2 ** 2, c='black',
                    marker='^', label='train')
        plt.scatter(species.pts_test['dd long'], species.pts_test['dd lat'],
                    s=2 ** 2, c='black',
                    marker='x', label='test')
        plt.legend()
        plt.title(species.name)
        plt.axis('equal')

        # Compute AUC with regards to background points
        pred_background = Z[background_points[0], background_points[1]]
        pred_test = clf.decision_function((species.cov_test - mean)
                                          / std)[:, 0]
        scores = np.r_[pred_test, pred_background]
        y = np.r_[np.ones(pred_test.shape), np.zeros(pred_background.shape)]
        fpr, tpr, thresholds = metrics.roc_curve(y, scores)
        roc_auc = metrics.auc(fpr, tpr)
        plt.text(-35, -70, "AUC: %.3f" % roc_auc, ha="right")
        print("\n Area under the ROC curve : %f" % roc_auc)

    print("\ntime elapsed: %.2fs" % (time() - t0))


plot_species_distribution()
plt.show()

## [Tensorflow](https://www.tensorflow.org/)

Google's latest ML library

Also see [TF-Slim](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim)

In [ ]:
from IPython import display
display.HTML('<iframe src="http://playground.tensorflow.org/" height=500 width=1024>')

Slide materials adapted from [Tom Augspurger](https://github.com/TomAugspurger/pydata-chi-h2t/blob/master/6-StatsAndML.ipynb)